In [64]:
# Imports
import os
import tensorflow as tf
import numpy as np
import seaborn as sns
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

In [65]:
# Caminhos locais (modifique conforme necessário)
train_dir = './data/images/'
test_dir = './data/tests/'

In [66]:
# Semente para reprodutibilidade
tf.random.set_seed(42)

# Carregamento do dataset com divisão automática treino/validação
train_dataset = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(32, 32),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(32, 32),
    batch_size=32
)

# Dataset de teste com rótulos
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(32, 32),
    batch_size=32,
    shuffle=False,
    label_mode='int'
)

Found 42 files belonging to 2 classes.
Using 34 files for training.
Found 42 files belonging to 2 classes.
Using 8 files for validation.
Found 10 files belonging to 2 classes.


In [67]:
# Normalização
normalization_layer = Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Definir modelo CNN
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # Ajuste conforme número de classes
])

In [68]:
# Compilar o modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [71]:
# EarlyStopping para evitar overfitting
es = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

# Resumo do modelo
model.summary()

# Treinamento
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=80,
    callbacks=[es]
)

# Avaliação no dataset de teste
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │         147,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 501,128 (1.91 MB)

 Trainable params: 167,042 (652.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 334,086 (1.27 MB)

Epoch 1/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step - accuracy: 0.6501 - loss: 0.8247 - val_accuracy: 0.7500 - val_loss: 0.5492
Epoch 2/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step - accuracy: 0.6605 - loss: 0.6364 - val_accuracy: 0.7500 - val_loss: 0.5815
Epoch 3/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 379ms/step - accuracy: 0.7298 - loss: 0.5496 - val_accuracy: 0.2500 - val_loss: 0.7344
Epoch 4/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step - accuracy: 0.6397 - loss: 0.6311 - val_accuracy: 0.2500 - val_loss: 0.8055
Epoch 5/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step - accuracy: 0.3995 - loss: 0.6870 - val_accuracy: 0.2500 - val_loss: 0.8348
Epoch 6/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step - accuracy: 0.4400 - loss: 0.6666 - val_accuracy: 0.7500 - val_loss: 0.6427
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.5000 - loss: 0.9400
Test Loss: 0.940004825592041
Test Accuracy: 0.5
